In [53]:
import os
import sys

import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

from core.services.backend_api_client import BackendAPIClient
from core.utils import load_dict_from_yaml

In [54]:
# Load all configuration from config folders
# The config can generated first using hummingbot and copied to the config folder
top_configs = [
    load_dict_from_yaml(file_name='macd_bb_1.yml', folder=root_path+"/config"),
    load_dict_from_yaml(file_name='macd_bb_2.yml', folder=root_path+"/config"),
]

In [55]:
# Use the backend API client to add the strategy
client = BackendAPIClient()
# The total balance that will used to run the asym_babel strategy
total_quote_balance = 100

# Setup config amount based on the strategy
# The type is { "id_from_config_file": amt_percentage } 
asym_babel_each_strat_balance = {
     "macd_bb_1": 0.8,
     "macd_bb_2": 0.2
}

# Modified total amount quote balance for each strategy based on the asym_babel_each_strat_balance
for config in top_configs:
     config["total_amount_quote"] = total_quote_balance * asym_babel_each_strat_balance[config["id"]]
     await client.add_controller_config(config)

In [56]:
# Deploy the strategy with multiple controllers
await client.deploy_script_with_controllers(
    bot_name="test_2_strategy",
    controller_configs=[config["id"] + ".yml" for config in top_configs],
    image_name="hummingbot/hummingbot:latest",
    max_global_drawdown=50,
    max_controller_drawdown=20,
)

{'success': True,
 'message': 'Instance hummingbot-test_2_strategy-2025.04.25_03.42 created successfully.'}